## Setup

In this notebook we solve a **steady, incompressible Navier–Stokes** problem with a Brinkman term so that solid regions can be embedded directly in the domain. This formulation can be usedto solve a pure fluid flow case and in topology optimization.

### Governing equations  

$$
\rho\,\mathbf{u}\,\cdot\nabla\mathbf{u}
-\nabla\!\cdot\!\Bigl[\mu\bigl(\nabla\mathbf{u}+\nabla\mathbf{u}^{\mathrm T}\bigr)\Bigr]
+\nabla p
+\alpha\,\mathbf{u}
= \mathbf{0}
\qquad\text{in } \Omega,
$$  

$$
\nabla\!\cdot\!\mathbf{u}=0
\qquad\text{in } \Omega.
$$  

Here  

* $\mathbf u=(u_x,u_y)$ velocity  
* $p$ pressure  
* $\rho$ density  
* $\mu$ dynamic viscosity  
* $\alpha$ Brinkman penalisation that damps flow in solid zones.

### Boundary conditions  

$$
\begin{aligned}
\mathbf{u} &= \mathbf{u}_{\text{in}}
&&\text{on } \Gamma_{\text{in}}\quad\text{(prescribed inflow)},\\[4pt]
-p\,\mathbf{n} + \mu\bigl(\nabla\mathbf{u}+\nabla\mathbf{u}^{\mathrm T}\bigr)\mathbf{n} &= \mathbf{0}
&&\text{on } \Gamma_{\text{out}}\quad\text{(zero-normal-stress outflow)},\\[4pt]
\mathbf{u} &= \mathbf{0}
&&\text{on } \Gamma_{\text{wall}}\quad\text{(no-slip walls)}.
\end{aligned}
$$  



---

### Test case: channel-flow benchmark  

<img src="../../figures/channel.png" alt="Channel" style="width: 500px;"/>

A fully developed laminar flow enters a straight 2-D channel and exits
through a zeero pressure outlet.  
The domain is discretised with bilinear quadrilateral elements; nodal
velocities are paired with nodal pressures.

We integrate the incompressible Navier–Stokes equations with a damped
Newton iteration and a linear solver, then verify the
centre-line velocity profile and overall pressure drop against classical
Poiseuille-flow theory.


In [ ]:
import numpy as np
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt

import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.utils as _utils
import toflux.src.bc as _bc
import toflux.src.fe_fluid as _fea
import toflux.src.solver as _solv
import toflux.src.viz as _viz

jax.config.update("jax_enable_x64", True)
plt.rcParams.update(_viz.high_res_plot_settings)

_Field = _fea.FluidField

### Mesh  

Below we build the canonical **channel-flow** benchmark commonly used in fluid-dynamics.

#### Geometry  
A simple straight channel is described in a json and passed to the geometry. This geometry is passed to the mesher.

#### Discretisation  
* The domain $(\Omega)$ is partitioned into **bilinear quadrilateral (Q1) elements**.  
* Both primary fields—velocity $(\mathbf u=(u_x,u_y))$ **and** pressure (p\)—are interpolated with the same Q1 shape functions (equal-order Q1/Q1 formulation).

#### Numerical integration  
Element contributions (mass, convection, brinkman, viscous and stability terms) are evaluated with a $(2 \times 2)$ Gauss quadrature.

This mesh serves as the baseline for the finite-element Navier–Stokes solve and for any subsequent density-based topology-optimisation runs.


In [ ]:
geom = _geom.BrepGeometry(brep_file="toflux/brep/channel_flow.json")

mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=200,
  nely_desired=40,
  dofs_per_node=3,
  gauss_order=2,
)

_viz.plot_brep(geom)

### Material  

For this fluid demonstration we keep the parameters simple and dimension-consistent:

| Symbol | Meaning | Value |
|--------|---------|-------|
| $(\rho)$ | Mass density | **1.2 kg m⁻³** (air at room temperature) |
| $(\mu)$ | Dynamic viscosity | **1.8 × 10⁻⁵ Pa s** |

We also prepare the *inverse permeability* field used in the Brinkman term.  
A RAMP interpolation converts the design variable $(\gamma\in[0,1])$ into the damping coefficient $(\alpha)$. In this fluid flow problem we enforce 0. brinkman penalty to recover the Navier Stokes equation. However, in the subsequent topology optimization the brinkman penalty value damps fluid flow in solid.


In [ ]:
mat_params = _mat.FluidMaterial(
  mass_density=1.2,
  dynamic_viscosity=1.8e-5,
)
mat_frac = 0.0 * jnp.ones((mesh.num_elems,)).astype(jnp.float64)
inv_permeability_ext = _utils.Extent(min=0.0, max=122.5)
brinkman_penalty = _mat.compute_ramp_interpolation(
  prop=mat_frac,
  ramp_penalty=8.0,
  prop_ext=inv_permeability_ext,
)

# Boundray Conditions


| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet (left vertical edge)** | Dirichlet | $(u = U_\text{c})$, $(v = 0)$ |
| **Outlet (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a channel Reynolds number of 100:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 100,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{channel height}.
$$


In [ ]:
reynolds_num = 100.0
char_velocity = reynolds_num * mat_params.kinematic_viscosity / (mesh.bounding_box.ly)

In [ ]:
face_tol = mesh.elem_size[0] * 0.5

# inlet condition
inlet_faces = _bc.identify_faces(mesh, edges=[geom.edges[3]], tol=face_tol)
n = len(inlet_faces)
u_vel = (_Field.U_VEL, char_velocity * jnp.ones(n))
v_vel = (_Field.V_VEL, jnp.zeros(n))
inlet_face_val = [u_vel, v_vel]

# outlet condition
outlet_faces = _bc.identify_faces(mesh, edges=[geom.edges[1]], tol=face_tol)
n = len(outlet_faces)
v_vel = (_Field.V_VEL, jnp.zeros(n))
pres = (_Field.PRESSURE, jnp.zeros(n))
outlet_face_val = [v_vel, pres]

# top condition
top_faces = _bc.identify_faces(mesh, edges=[geom.edges[2]], tol=face_tol)
n = len(top_faces)
u_vel = (_Field.U_VEL, jnp.zeros(n))
v_vel = (_Field.V_VEL, jnp.zeros(n))
top_face_val = [u_vel, v_vel]

# bottom condition
bottom_faces = _bc.identify_faces(mesh, edges=[geom.edges[0]], tol=face_tol)
n = len(bottom_faces)
u_vel = (_Field.U_VEL, jnp.zeros(n))
v_vel = (_Field.V_VEL, jnp.zeros(n))
bottom_face_val = [u_vel, v_vel]

inlet_bc = _bc.DirichletBC(elem_faces=inlet_faces, values=inlet_face_val, name="inlet")
outlet_bc = _bc.DirichletBC(
  elem_faces=outlet_faces, values=outlet_face_val, name="outlet"
)
top_bc = _bc.DirichletBC(elem_faces=top_faces, values=top_face_val, name="top")
bottom_bc = _bc.DirichletBC(
  elem_faces=bottom_faces, values=bottom_face_val, name="bottom"
)

bc_list = [inlet_bc, outlet_bc, top_bc, bottom_bc]
bc = _bc.process_boundary_conditions(bc_list, mesh)

_viz.plot_bc(bc_list, mesh)

# Solver


The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'.


In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.SCIPY_SPARSE,
    "rtol": 1.0e-3,
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-4},
}

# Initialize

We initialize the FEA solver with the mesh, material, boundary conditions and solver settings.

In [ ]:
fea = _fea.FluidSolver(mesh, bc, mat_params, solver_settings=solver_settings)

# Solve
We provide an inital guess of the solution with the Dirichlet condition enforced.

In [ ]:
init_press_vel = jnp.zeros((mesh.num_dofs,))
init_press_vel = init_press_vel.at[bc["fixed_dofs"]].set(bc["dirichlet_values"])

press_vel = _solv.modified_newton_raphson_solve(fea, init_press_vel, brinkman_penalty)

# Plot

We visualize the horizontal velocity here.

In [ ]:
press_vel_elem = press_vel[mesh.elem_dof_mat]
press_elem = np.mean(press_vel_elem[:, 0::3], axis=1)
u_vel_elem = np.mean(press_vel_elem[:, 1::3], axis=1)
v_vel_elem = np.mean(press_vel_elem[:, 2::3], axis=1)
vel_elem_mag = np.sqrt(u_vel_elem**2 + v_vel_elem**2)
_, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(mesh=mesh, field=vel_elem_mag, ax=ax, colorbar=True)
ax.set_title("Velocity Magnitude")

_, ax = plt.subplots(1, 1)
ax = _viz.plot_grid_mesh(mesh=mesh, field=press_elem, ax=ax, colorbar=True)
ax.set_title("Pressure")
plt.show()